## Vector Database Setup

In [27]:
from dotenv import load_dotenv

load_dotenv()

True

In [28]:
import json

def json_print(data):
    print(json.dumps(data, indent=2))

### Download sample data

In [33]:
import requests
import json

# Download the data
resp = requests.get('https://raw.githubusercontent.com/weaviate-tutorials/quickstart/main/data/jeopardy_tiny.json')
data = json.loads(resp.text)

print(type(data), len(data))

def json_print(data):
    print(json.dumps(data, indent=2))

json_print(data[0])

<class 'list'> 10
{
  "Category": "SCIENCE",
  "Question": "This organ removes excess glucose from the blood & stores it as glycogen",
  "Answer": "Liver"
}


### Create an embedded instance of Weaviate vector database

In [57]:
import os

import weaviate
from weaviate.embedded import EmbeddedOptions


client = weaviate.Client(
    embedded_options=EmbeddedOptions(),
    additional_headers={
        "X-OpenAI-Api-Key": os.environ['OPENAI_API_KEY']
    }
)

/var/folders/6c/jszt_7h51gq3vsv4hyvd0xfh0000gp/T/ipykernel_86614/86879308.py:7: DeprecationWarning: 
Python client v3 `weaviate.Client(...)` connections and methods are deprecated and will
            be removed by 2024-11-30.

            Upgrade your code to use Python client v4 `weaviate.WeaviateClient` connections and methods.
                - For Python Client v4 usage, see: https://weaviate.io/developers/weaviate/client-libraries/python
                - For code migration, see: https://weaviate.io/developers/weaviate/client-libraries/python/v3_v4_migration

            If you have to use v3 code, install the v3 client and pin the v3 dependency in your requirements file: `weaviate-client>=3.26.7;<4.0.0`
  client = weaviate.Client(


{"level":"warning","msg":"prop len tracker file /Users/arslanhaider/.local/share/weaviate/question/QF073Ms4dP0J/proplengths does not exist, creating new tracker","time":"2024-10-05T01:20:03+05:00"}
{"action":"hnsw_prefill_cache_async","level":"info","msg":"not waiting for vector cache prefill, running in background","time":"2024-10-05T01:20:03+05:00","wait_for_cache_prefill":false}
{"level":"info","msg":"Created shard question_QF073Ms4dP0J in 4.356458ms","time":"2024-10-05T01:20:03+05:00"}
{"action":"hnsw_vector_cache_prefill","count":1000,"index_id":"main","level":"info","limit":1000000000000,"msg":"prefilled vector cache","time":"2024-10-05T01:20:03+05:00","took":90542}


In [39]:
json_print(client.get_meta())

{
  "hostname": "http://127.0.0.1:8079",
  "modules": {
    "generative-openai": {
      "documentationHref": "https://platform.openai.com/docs/api-reference/completions",
      "name": "Generative Search - OpenAI"
    },
    "qna-openai": {
      "documentationHref": "https://platform.openai.com/docs/api-reference/completions",
      "name": "OpenAI Question & Answering Module"
    },
    "ref2vec-centroid": {},
    "reranker-cohere": {
      "documentationHref": "https://txt.cohere.com/rerank/",
      "name": "Reranker - Cohere"
    },
    "text2vec-cohere": {
      "documentationHref": "https://docs.cohere.ai/embedding-wiki/",
      "name": "Cohere Module"
    },
    "text2vec-huggingface": {
      "documentationHref": "https://huggingface.co/docs/api-inference/detailed_parameters#feature-extraction-task",
      "name": "Hugging Face Module"
    },
    "text2vec-openai": {
      "documentationHref": "https://platform.openai.com/docs/guides/embeddings/what-are-embeddings",
      "nam

### Create Question Collection

In [40]:
class_obj = {
    "class": "Question",
    "vectorizer": "text2vec-openai",
    "moduleConfig": {
        "text2vec-openai": {
            "model": "ada",
            "modelVersion": "002",
            "type": "text"
        }
    }
}

client.schema.create_class(class_obj)

{"level":"warning","msg":"prop len tracker file /Users/arslanhaider/.local/share/weaviate/question/ctWyf2hgZavq/proplengths does not exist, creating new tracker","time":"2024-10-05T01:15:27+05:00"}
{"action":"hnsw_prefill_cache_async","level":"info","msg":"not waiting for vector cache prefill, running in background","time":"2024-10-05T01:15:27+05:00","wait_for_cache_prefill":false}
{"level":"info","msg":"Created shard question_ctWyf2hgZavq in 8.16575ms","time":"2024-10-05T01:15:27+05:00"}
{"action":"hnsw_vector_cache_prefill","count":1000,"index_id":"main","level":"info","limit":1000000000000,"msg":"prefilled vector cache","time":"2024-10-05T01:15:27+05:00","took":176834}


### Load sample data and generate vector embeddings

In [41]:
json_print(data[0])

{
  "Category": "SCIENCE",
  "Question": "This organ removes excess glucose from the blood & stores it as glycogen",
  "Answer": "Liver"
}


In [42]:
with client.batch.configure(batch_size=5) as batch:
    for i, d in enumerate(data):
        
        print(f"importing question: {i+1}")
        
        properties = {
            "answer": d["Answer"],
            "question": d["Question"],
            "category": d["Category"],
        }
        
        batch.add_data_object(
            data_object=properties,
            class_name="Question"
        )

importing question: 1
importing question: 2
importing question: 3
importing question: 4
importing question: 5
importing question: 6
importing question: 7
importing question: 8
importing question: 9
importing question: 10


In [43]:
count = client.query.aggregate("Question").with_meta_count().do()
json_print(count)

{
  "data": {
    "Aggregate": {
      "Question": [
        {
          "meta": {
            "count": 10
          }
        }
      ]
    }
  }
}


### Extract the vector that represents each question

In [44]:
result = (client.query
          .get("Question", ["category", "question", "answer"])
          .with_additional("vector")
          .with_limit(1)
          .do())

json_print(result)

{
  "data": {
    "Get": {
      "Question": [
        {
          "_additional": {
            "vector": [
              -0.010680359,
              -0.010680359,
              -0.013921662,
              -0.028560663,
              0.00032898728,
              0.017654475,
              -0.019261843,
              -0.005688885,
              -0.006419507,
              -0.005838331,
              0.013164473,
              0.010394753,
              0.018610924,
              -0.017056692,
              -0.017627906,
              0.01206854,
              0.017986575,
              0.017388795,
              0.017362226,
              0.016259652,
              0.011092164,
              0.004672657,
              0.004171185,
              -0.010341616,
              0.03514954,
              -0.010627223,
              0.011470759,
              -0.0060973694,
              -0.025943708,
              0.0008975022,
              0.0057752314,
              -0.012978496,
          

In [45]:
response = (
    client.query
    .get("Question",["question", "answer","category"])
    .with_near_text({"concepts": "biology"})
    .with_additional('distance')
    .with_limit(2)
    .do()
)

json_print(response)

{
  "data": {
    "Get": {
      "Question": [
        {
          "_additional": {
            "distance": 0.19779843
          },
          "answer": "DNA",
          "category": "SCIENCE",
          "question": "In 1953 Watson & Crick built a model of the molecular structure of this, the gene-carrying substance"
        },
        {
          "_additional": {
            "distance": 0.20255458
          },
          "answer": "species",
          "category": "SCIENCE",
          "question": "2000 news: the Gunnison sage grouse isn't just another northern sage grouse, but a new one of this classification"
        }
      ]
    }
  }
}


In [46]:
response = (
    client.query
    .get("Question", ["question", "answer"])
    .with_near_text({"concepts": ["animals"]})
    .with_limit(10)
    .with_additional(["distance"])
    .do()
)

json_print(response)

{
  "data": {
    "Get": {
      "Question": [
        {
          "_additional": {
            "distance": 0.19058836
          },
          "answer": "Elephant",
          "question": "It's the only living mammal in the order Proboseidea"
        },
        {
          "_additional": {
            "distance": 0.19264841
          },
          "answer": "the nose or snout",
          "question": "The gavial looks very much like a crocodile except for this bodily feature"
        },
        {
          "_additional": {
            "distance": 0.20513117
          },
          "answer": "Antelope",
          "question": "Weighing around a ton, the eland is the largest species of this animal in Africa"
        },
        {
          "_additional": {
            "distance": 0.21583492
          },
          "answer": "species",
          "question": "2000 news: the Gunnison sage grouse isn't just another northern sage grouse, but a new one of this classification"
        },
        {
    

In [47]:
response = (
    client.query
    .get("Question", ["question", "answer"])
    .with_near_text({"concepts": ["animals"], "distance": 0.24})
    .with_limit(10)
    .with_additional(["distance"])
    .do()
)

json_print(response)

{
  "data": {
    "Get": {
      "Question": [
        {
          "_additional": {
            "distance": 0.19058836
          },
          "answer": "Elephant",
          "question": "It's the only living mammal in the order Proboseidea"
        },
        {
          "_additional": {
            "distance": 0.19264841
          },
          "answer": "the nose or snout",
          "question": "The gavial looks very much like a crocodile except for this bodily feature"
        },
        {
          "_additional": {
            "distance": 0.20513117
          },
          "answer": "Antelope",
          "question": "Weighing around a ton, the eland is the largest species of this animal in Africa"
        },
        {
          "_additional": {
            "distance": 0.21583492
          },
          "answer": "species",
          "question": "2000 news: the Gunnison sage grouse isn't just another northern sage grouse, but a new one of this classification"
        },
        {
    

## Vector Databases support for CRUD operations

### Create

In [48]:
#Create an object
object_uuid = client.data_object.create(
    data_object={
        'question':"Leonardo da Vinci was born in this country.",
        'answer': "Italy",
        'category': "Culture"
    },
    class_name="Question"
 )

In [ ]:
print(object_uuid)

### Read

In [49]:
data_object = client.data_object.get_by_id(object_uuid, class_name="Question")
json_print(data_object)

{
  "class": "Question",
  "creationTimeUnix": 1728072977780,
  "id": "209524d4-41ff-4399-9936-2cc2b5ce7039",
  "lastUpdateTimeUnix": 1728072977780,
  "properties": {
    "answer": "Italy",
    "category": "Culture",
    "question": "Leonardo da Vinci was born in this country."
  },
  "vectorWeights": null
}


In [50]:
data_object = client.data_object.get_by_id(
    object_uuid,
    class_name='Question',
    with_vector=True
)

json_print(data_object)

{
  "class": "Question",
  "creationTimeUnix": 1728072977780,
  "id": "209524d4-41ff-4399-9936-2cc2b5ce7039",
  "lastUpdateTimeUnix": 1728072977780,
  "properties": {
    "answer": "Italy",
    "category": "Culture",
    "question": "Leonardo da Vinci was born in this country."
  },
  "vector": [
    0.022554897,
    -0.013080306,
    -0.003111847,
    -0.047769327,
    -0.0038806186,
    0.0120318355,
    -0.017440408,
    -0.004030857,
    -0.0045998436,
    -0.03459952,
    0.010471917,
    0.025649164,
    -0.0031725815,
    -0.0072817537,
    0.010337661,
    -0.0014991846,
    0.012971623,
    0.0054757,
    0.03961172,
    -0.007940244,
    0.0023318874,
    0.021173986,
    0.01171218,
    -0.010983366,
    0.0051624374,
    0.0018300281,
    0.016532587,
    -0.03211899,
    0.014934308,
    -0.03188884,
    -0.0078443475,
    0.0007491927,
    -0.0099668605,
    -0.013374389,
    -0.018782962,
    -0.014486791,
    -0.007678127,
    -0.028180836,
    0.00824072,
    0.0153818

### Update

In [51]:
client.data_object.update(
    uuid=object_uuid,
    class_name="Question",
    data_object={
        'answer':"Florence, Italy"
    })

In [52]:
data_object = client.data_object.get_by_id(
    object_uuid,
    class_name='Question',
)

json_print(data_object)

{
  "class": "Question",
  "creationTimeUnix": 1728072977780,
  "id": "209524d4-41ff-4399-9936-2cc2b5ce7039",
  "lastUpdateTimeUnix": 1728072986978,
  "properties": {
    "answer": "Florence, Italy",
    "category": "Culture",
    "question": "Leonardo da Vinci was born in this country."
  },
  "vectorWeights": null
}


### Delete

In [53]:
json_print(client.query.aggregate("Question").with_meta_count().do())

{
  "data": {
    "Aggregate": {
      "Question": [
        {
          "meta": {
            "count": 11
          }
        }
      ]
    }
  }
}


In [54]:
client.data_object.delete(uuid=object_uuid, class_name="Question")

In [55]:
json_print(client.query.aggregate("Question").with_meta_count().do())

{
  "data": {
    "Aggregate": {
      "Question": [
        {
          "meta": {
            "count": 10
          }
        }
      ]
    }
  }
}
